<a href="https://colab.research.google.com/github/YaPineiro/Fin420/blob/main/YP_week_1_session_1_alternative_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is the average on and off-peak real-time fuel mix on the [NYISO](https://www.nyiso.com/) grid?

Power is provided to our electric grid from a variety of sources.  Broadly the types can be classified as:

*   Base Load (for example Hydro, Nuclear)
*   Intermediate Load (Natural Gas)
*   Peakers (Fuel Oil and other inefficient plants)

So as the amount of demand changes throughout the day, the proportion of electricity from each source will change.  As power demand (also known as `load`) approaches is minimum for the day, proportionally more electricity will be produced by base load plants.  As electricity demand increases, proportionally more power will be produced from intermediate and peaking sources. Also, since wind has a priority dispatch, wind will have a larger proportion during the more windy times of day.



## On Versus Off Peak

We refer to the times with the higher expected load as `on-peak`.  Days and times with lower expected power demand is `off-peak`.  These terms are written into futures and OTC derivative contracts, and are thus meaningful.

### On-Peak

-  Weekdays from 7am to 11pm

### Off-Peak

- Weekdays from 11pm to 7am.
- Weekends


# Analysis

Below we are going to use Python and Pandas to calculate the average real-time fuel mixes.  We'll also us the `os` module to interact with our file system (add and delete files and directories).  Below are the packages we will use:



In [1]:
import pandas as pd
import os
import sys
import urllib.request
import glob
import zipfile
import plotly.express as px

NYISO provides fuel mix data sampled at a five minute frequency for each month here: `http://mis.nyiso.com/public/csv/rtfuelmix/YYYYMMDDrtfuelmix.csv`  where `YYYYMMDD` is the date.

For example, let's import the data for October 2020:

But note this is a zip file.  So we want to unzip and merge all the files before creating our pandas dataframe.

# Python `os` vs System Utilities

Below we'll use both Linux system utilities, as well as the Python equivalents provided by the `os` module.  Remember we can interact with the file system via a [bash shell](https://en.wikipedia.org/wiki/Bash_%28Unix_shell%29). To do so we prepend our command with `!`.

In [2]:
!echo "Our shell is $SHELL"

Our shell is /bin/bash


Where are we in the system?

In [36]:
!pwd

/content


What else is in this directory?  You may want to use this command later to verify a file or directory has been created.

In [37]:
!ls ./

merged_data  raw_data  sample_data


## In Python using `os`

Instead of using the shell, we can also use the Python `os` module methods to interact with the system.  Note we imported the module above with `import os`. To see all the *attributes* and *methods* provided by the `os` module we can do:

In [38]:
print(dir(os))

['CLD_CONTINUED', 'CLD_DUMPED', 'CLD_EXITED', 'CLD_KILLED', 'CLD_STOPPED', 'CLD_TRAPPED', 'DirEntry', 'EFD_CLOEXEC', 'EFD_NONBLOCK', 'EFD_SEMAPHORE', 'EX_CANTCREAT', 'EX_CONFIG', 'EX_DATAERR', 'EX_IOERR', 'EX_NOHOST', 'EX_NOINPUT', 'EX_NOPERM', 'EX_NOUSER', 'EX_OK', 'EX_OSERR', 'EX_OSFILE', 'EX_PROTOCOL', 'EX_SOFTWARE', 'EX_TEMPFAIL', 'EX_UNAVAILABLE', 'EX_USAGE', 'F_LOCK', 'F_OK', 'F_TEST', 'F_TLOCK', 'F_ULOCK', 'GRND_NONBLOCK', 'GRND_RANDOM', 'GenericAlias', 'MFD_ALLOW_SEALING', 'MFD_CLOEXEC', 'MFD_HUGETLB', 'MFD_HUGE_16GB', 'MFD_HUGE_16MB', 'MFD_HUGE_1GB', 'MFD_HUGE_1MB', 'MFD_HUGE_256MB', 'MFD_HUGE_2GB', 'MFD_HUGE_2MB', 'MFD_HUGE_32MB', 'MFD_HUGE_512KB', 'MFD_HUGE_512MB', 'MFD_HUGE_64KB', 'MFD_HUGE_8MB', 'MFD_HUGE_MASK', 'MFD_HUGE_SHIFT', 'Mapping', 'MutableMapping', 'NGROUPS_MAX', 'O_ACCMODE', 'O_APPEND', 'O_ASYNC', 'O_CLOEXEC', 'O_CREAT', 'O_DIRECT', 'O_DIRECTORY', 'O_DSYNC', 'O_EXCL', 'O_FSYNC', 'O_LARGEFILE', 'O_NDELAY', 'O_NOATIME', 'O_NOCTTY', 'O_NOFOLLOW', 'O_NONBLOCK', 'O_P

We will only need a few of these however.  For example, using the following we can create a directory within the present directory, named `raw_data`.

In [39]:
os.makedirs("./raw_data")

FileExistsError: [Errno 17] File exists: './raw_data'

The equivalent in teh shell would be the following.  Note it is commented so it doesn't run.  It would generate an error because the directory already exists, thanks to the above line.

In [7]:
#!mkdir raw_data

## Download Data from NYISO

We can download the file either using python itself or via the `wget` command line utility (program) which is ubiquitous on *nix systems.  If you are going to work later on Windows systems, you may want to use the Python method (note file paths use `\` on windows rather than the *nix `/`.)

In [70]:
year_month = "202009"
urllib.request.urlretrieve("http://mis.nyiso.com/public/csv/rtfuelmix/" + year_month + "01rtfuelmix_csv.zip",  "./raw_data/"+ year_month + "data.zip")

('./raw_data/202009data.zip', <http.client.HTTPMessage at 0x7fca3f52f0d0>)

In [71]:
#!wget --verbose -O ./raw_data/202009_data.zip http://mis.nyiso.com/public/csv/rtfuelmix/20200901rtfuelmix_csv.zip

## Download Data from Github

Only use the below if NYISO website is down.  Otherwise skip.

Within Python method:

In [ ]:
year_month = "202009"
urllib.request.urlretrieve("https://github.com/FinancialMarkets/nyiso_fuel_mix_data/blob/master/" + year_month + "data.zip?raw=true",  "./raw_data/"+ year_month + "data.zip")

('./raw_data/202009data.zip', <http.client.HTTPMessage at 0x7f8b24b382b0>)

In [ ]:
!ls ./raw_data/

20200901rtfuelmix.csv  20200912rtfuelmix.csv  20200923rtfuelmix.csv
20200902rtfuelmix.csv  20200913rtfuelmix.csv  20200924rtfuelmix.csv
20200903rtfuelmix.csv  20200914rtfuelmix.csv  20200925rtfuelmix.csv
20200904rtfuelmix.csv  20200915rtfuelmix.csv  20200926rtfuelmix.csv
20200905rtfuelmix.csv  20200916rtfuelmix.csv  20200927rtfuelmix.csv
20200906rtfuelmix.csv  20200917rtfuelmix.csv  20200928rtfuelmix.csv
20200907rtfuelmix.csv  20200918rtfuelmix.csv  20200929rtfuelmix.csv
20200908rtfuelmix.csv  20200919rtfuelmix.csv  20200930rtfuelmix.csv
20200909rtfuelmix.csv  20200920rtfuelmix.csv  202009_data.zip
20200910rtfuelmix.csv  20200921rtfuelmix.csv  202009data.zip
20200911rtfuelmix.csv  20200922rtfuelmix.csv


Using `wget`.

In [ ]:
!wget --verbose -O ./raw_data/202009data.zip https://github.com/FinancialMarkets/nyiso_fuel_mix_data/blob/master/202009data.zip?raw=true

--2020-12-30 00:13:43--  https://github.com/FinancialMarkets/nyiso_fuel_mix_data/blob/master/202009data.zip?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/FinancialMarkets/nyiso_fuel_mix_data/raw/master/202009data.zip [following]
--2020-12-30 00:13:43--  https://github.com/FinancialMarkets/nyiso_fuel_mix_data/raw/master/202009data.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FinancialMarkets/nyiso_fuel_mix_data/master/202009data.zip [following]
--2020-12-30 00:13:44--  https://raw.githubusercontent.com/FinancialMarkets/nyiso_fuel_mix_data/master/202009data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubus

## Unzip Data

Unzip via Python.  The following uses a [with](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) statement to wrap our code in a [Context Manager](https://docs.python.org/3/reference/datamodel.html#context-managers).  This manages the opening and closing of the file.  We don't have to worry about how this works at this point---it is just a way to ensure the file is closed even if our code fails for some reason.

In [72]:
year_month = "202009"

with zipfile.ZipFile("./raw_data/" + year_month + "data.zip","r") as zip_ref:
    zip_ref.extractall("./raw_data")

Alternatively, unzip using the `unzip` command line utility:

In [ ]:
#unzip ./raw_data/202009data.zip

In [73]:
!ls ./raw_data/

20200901rtfuelmix.csv  20200909rtfuelmix.csv  20200917rtfuelmix.csv  20200925rtfuelmix.csv
20200902rtfuelmix.csv  20200910rtfuelmix.csv  20200918rtfuelmix.csv  20200926rtfuelmix.csv
20200903rtfuelmix.csv  20200911rtfuelmix.csv  20200919rtfuelmix.csv  20200927rtfuelmix.csv
20200904rtfuelmix.csv  20200912rtfuelmix.csv  20200920rtfuelmix.csv  20200928rtfuelmix.csv
20200905rtfuelmix.csv  20200913rtfuelmix.csv  20200921rtfuelmix.csv  20200929rtfuelmix.csv
20200906rtfuelmix.csv  20200914rtfuelmix.csv  20200922rtfuelmix.csv  20200930rtfuelmix.csv
20200907rtfuelmix.csv  20200915rtfuelmix.csv  20200923rtfuelmix.csv  202009data.zip
20200908rtfuelmix.csv  20200916rtfuelmix.csv  20200924rtfuelmix.csv


We'll stop using shell code here.  But note often the shell code is more simple than Python.  Also, you can call shell code from Python, so sometimes it may help to call shell code in a Python script.  The only downside to this is shell code is different between Windows and Linux/macOS.  Note, macOS uses [zsh](https://www.zsh.org/) which is compatible with BASH.

In [74]:
os.remove("./raw_data/" + year_month + "data.zip")

In [75]:
!ls ./raw_data/

20200901rtfuelmix.csv  20200909rtfuelmix.csv  20200917rtfuelmix.csv  20200925rtfuelmix.csv
20200902rtfuelmix.csv  20200910rtfuelmix.csv  20200918rtfuelmix.csv  20200926rtfuelmix.csv
20200903rtfuelmix.csv  20200911rtfuelmix.csv  20200919rtfuelmix.csv  20200927rtfuelmix.csv
20200904rtfuelmix.csv  20200912rtfuelmix.csv  20200920rtfuelmix.csv  20200928rtfuelmix.csv
20200905rtfuelmix.csv  20200913rtfuelmix.csv  20200921rtfuelmix.csv  20200929rtfuelmix.csv
20200906rtfuelmix.csv  20200914rtfuelmix.csv  20200922rtfuelmix.csv  20200930rtfuelmix.csv
20200907rtfuelmix.csv  20200915rtfuelmix.csv  20200923rtfuelmix.csv
20200908rtfuelmix.csv  20200916rtfuelmix.csv  20200924rtfuelmix.csv


In [76]:
os.makedirs("./merged_data/" + year_month + "/")

FileExistsError: [Errno 17] File exists: './merged_data/202009/'

In [77]:
!ls ./merged_data/

202009


## Append Into One DataFrame

The code below will create one large DataFrame by concatenating the smaller DataFrames from each file.  Line 1 created a list of all the files in the `raw_data` directory.  Line 3 creates an empty list called `fuel_list`.

Lines 5 through 7 then loops through all the files in the `raw_data` directory and adds them to `fuel_list`.  So `fuel_list` is a list of Pandas DataFrames.  The call to `concat` on line 9 then concatenates all the DataFrames in `fuel_list` into one DataFrame.

In [78]:
ff = os.listdir("./raw_data/")

fuel_list = []

for i in ff:
    temp = pd.read_csv("./raw_data/" + i)
    fuel_list.append(temp)

fuel = pd.concat(fuel_list)

In [79]:
print(ff)

['20200928rtfuelmix.csv', '20200906rtfuelmix.csv', '20200924rtfuelmix.csv', '20200930rtfuelmix.csv', '20200922rtfuelmix.csv', '20200919rtfuelmix.csv', '20200904rtfuelmix.csv', '20200912rtfuelmix.csv', '20200909rtfuelmix.csv', '20200929rtfuelmix.csv', '20200908rtfuelmix.csv', '20200920rtfuelmix.csv', '20200918rtfuelmix.csv', '20200923rtfuelmix.csv', '20200926rtfuelmix.csv', '20200902rtfuelmix.csv', '20200901rtfuelmix.csv', '20200913rtfuelmix.csv', '20200907rtfuelmix.csv', '20200910rtfuelmix.csv', '20200911rtfuelmix.csv', '20200915rtfuelmix.csv', '20200921rtfuelmix.csv', '20200917rtfuelmix.csv', '20200925rtfuelmix.csv', '20200927rtfuelmix.csv', '20200916rtfuelmix.csv', '20200903rtfuelmix.csv', '20200914rtfuelmix.csv', '20200905rtfuelmix.csv']


In [80]:
print(fuel_list)

[               Time Stamp Time Zone       Fuel Category  Gen MW
0     09/28/2020 00:05:00       EDT           Dual Fuel  2692.0
1     09/28/2020 00:05:00       EDT         Natural Gas  3869.0
2     09/28/2020 00:05:00       EDT             Nuclear  3473.0
3     09/28/2020 00:05:00       EDT  Other Fossil Fuels     1.0
4     09/28/2020 00:05:00       EDT    Other Renewables   197.0
...                   ...       ...                 ...     ...
2074  09/29/2020 00:00:00       EDT             Nuclear  3465.0
2075  09/29/2020 00:00:00       EDT  Other Fossil Fuels     1.0
2076  09/29/2020 00:00:00       EDT    Other Renewables   190.0
2077  09/29/2020 00:00:00       EDT                Wind  1217.0
2078  09/29/2020 00:00:00       EDT               Hydro  2246.0

[2079 rows x 4 columns],                Time Stamp Time Zone       Fuel Category  Gen MW
0     09/06/2020 00:05:00       EDT           Dual Fuel  3014.0
1     09/06/2020 00:05:00       EDT         Natural Gas  2088.0
2     09/06/2

In [81]:
type(fuel_list)

list

In [82]:
fuel_list[10]

,Time Stamp,Time Zone,Fuel Category,Gen MW
0,09/08/2020 00:05:00,EDT,Dual Fuel,3168.0
1,09/08/2020 00:05:00,EDT,Natural Gas,3324.0
2,09/08/2020 00:05:00,EDT,Nuclear,4097.0
3,09/08/2020 00:05:00,EDT,Other Fossil Fuels,1.0
4,09/08/2020 00:05:00,EDT,Other Renewables,242.0
...,...,...,...,...
2081,09/09/2020 00:00:00,EDT,Nuclear,4098.0
2082,09/09/2020 00:00:00,EDT,Other Fossil Fuels,1.0
2083,09/09/2020 00:00:00,EDT,Other Renewables,247.0
2084,09/09/2020 00:00:00,EDT,Wind,156.0


In [83]:
type(fuel_list[4])

pandas.core.frame.DataFrame

So we see `fuel_list` is a list of Pandas DataFrame objects.

Now lets take a look at the size and types of the data.

In [84]:
fuel

,Time Stamp,Time Zone,Fuel Category,Gen MW
0,09/28/2020 00:05:00,EDT,Dual Fuel,2692.0
1,09/28/2020 00:05:00,EDT,Natural Gas,3869.0
2,09/28/2020 00:05:00,EDT,Nuclear,3473.0
3,09/28/2020 00:05:00,EDT,Other Fossil Fuels,1.0
4,09/28/2020 00:05:00,EDT,Other Renewables,197.0
...,...,...,...,...
2039,09/06/2020 00:00:00,EDT,Nuclear,4105.0
2040,09/06/2020 00:00:00,EDT,Other Fossil Fuels,1.0
2041,09/06/2020 00:00:00,EDT,Other Renewables,218.0
2042,09/06/2020 00:00:00,EDT,Wind,247.0


In [85]:
fuel.dtypes

,0
Time Stamp,object
Time Zone,object
Fuel Category,object
Gen MW,float64


Let's convert the `Time Stamp` into a `datetime` object.

In [86]:
fuel["Time Stamp"] = pd.to_datetime(fuel["Time Stamp"])

In [87]:
fuel.dtypes

,0
Time Stamp,datetime64[ns]
Time Zone,object
Fuel Category,object
Gen MW,float64


Here we call `set_index` to

In [88]:
fuel.set_index(fuel["Time Stamp"], inplace=True)

The original code was asking to delete "Time Stamp" Column instead of the "Time Zone" column, so I was getting error when running the codes to obtain the averages. I corrected the code to drop the "Time Zone" code and all the below codes worked.

In [89]:
fuel.drop(columns="Time Zone", inplace=True)

In [90]:
off_peak = fuel.between_time('23:00:00', '06:59:59')

In [91]:
on_peak = fuel.between_time('07:00:00', '22:59:59')

In [92]:
off_peak

,Time Stamp,Fuel Category,Gen MW
Time Stamp,,,
2020-09-28 00:05:00,2020-09-28 00:05:00,Dual Fuel,2692.0
2020-09-28 00:05:00,2020-09-28 00:05:00,Natural Gas,3869.0
2020-09-28 00:05:00,2020-09-28 00:05:00,Nuclear,3473.0
2020-09-28 00:05:00,2020-09-28 00:05:00,Other Fossil Fuels,1.0
2020-09-28 00:05:00,2020-09-28 00:05:00,Other Renewables,197.0
...,...,...,...
2020-09-06 00:00:00,2020-09-06 00:00:00,Nuclear,4105.0
2020-09-06 00:00:00,2020-09-06 00:00:00,Other Fossil Fuels,1.0
2020-09-06 00:00:00,2020-09-06 00:00:00,Other Renewables,218.0


# Create Averages

In [93]:
off_peak_averages = off_peak.groupby(['Fuel Category']).mean()

In [94]:
off_peak_averages

,Time Stamp,Gen MW
Fuel Category,,
Dual Fuel,2020-09-15 18:56:50.065562624,2926.331263
Hydro,2020-09-15 18:56:50.065562624,2438.103520
Natural Gas,2020-09-15 18:56:50.065562624,2612.240511
Nuclear,2020-09-15 18:56:50.065562624,3767.163906
Other Fossil Fuels,2020-09-15 18:56:50.065562624,1.359558
Other Renewables,2020-09-15 18:56:50.065562624,234.669427
Wind,2020-09-15 18:56:50.065562624,509.129400


In [95]:
off_peak_averages['Gen_percent'] = off_peak_averages['Gen MW'] / sum(off_peak_averages['Gen MW'])

In [96]:
off_peak_averages

,Time Stamp,Gen MW,Gen_percent
Fuel Category,,,
Dual Fuel,2020-09-15 18:56:50.065562624,2926.331263,0.234313
Hydro,2020-09-15 18:56:50.065562624,2438.103520,0.195220
Natural Gas,2020-09-15 18:56:50.065562624,2612.240511,0.209163
Nuclear,2020-09-15 18:56:50.065562624,3767.163906,0.301639
Other Fossil Fuels,2020-09-15 18:56:50.065562624,1.359558,0.000109
Other Renewables,2020-09-15 18:56:50.065562624,234.669427,0.018790
Wind,2020-09-15 18:56:50.065562624,509.129400,0.040766


In [97]:
on_peak_averages = on_peak.groupby(['Fuel Category']).mean()

In [98]:
on_peak_averages['Gen_percent'] = on_peak_averages['Gen MW'] / sum(on_peak_averages['Gen MW'])

In [99]:
on_peak_averages

,Time Stamp,Gen MW,Gen_percent
Fuel Category,,,
Dual Fuel,2020-09-16 04:00:27.525710592,4247.893775,0.268019
Hydro,2020-09-16 04:00:27.525710592,3238.210758,0.204313
Natural Gas,2020-09-16 04:00:27.525710592,3938.297869,0.248485
Nuclear,2020-09-16 04:00:27.525710592,3760.147497,0.237245
Other Fossil Fuels,2020-09-16 04:00:27.525710592,21.394452,0.001350
Other Renewables,2020-09-16 04:00:27.525710592,242.823579,0.015321
Wind,2020-09-16 04:00:27.525710592,400.478518,0.025268


So, as we would expect, on-peak we have less nuclear and more natural gas and dual fuel generation.

## Visualize

In [100]:
fig = px.pie(off_peak_averages, values='Gen MW',  names=off_peak_averages.index, title="Off-Peak NYISO Fuel Mix")
fig.show()

In [101]:
fig = px.pie(on_peak_averages, values='Gen MW',  names=on_peak_averages.index, title="On-Peak NYISO Fuel Mix")
fig.show()

# Exercise

We made a mistake above.  We didn't take into account the weekend---between 7am and 11pm on Saturday and Sunday is classified as off-peak.

- Divide the data set into weekends and weekdays, and recalculate the proper average fuel mix on and off peak.
- You can use `pandas.DatetimeIndex.dayofweek` to get the day of the week: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.dayofweek.html

In [104]:
fuel["DayOfWeek"] = fuel.index.dayofweek
fuel["IsOffPeak"] = ((fuel.index.hour < 7) | (fuel.index.hour >= 23)) | (fuel["DayOfWeek"] >= 5)
on_peak = fuel[~fuel["IsOffPeak"]]
off_peak = fuel[fuel["IsOffPeak"]]

In [105]:
on_peak_averages = on_peak.groupby(['Fuel Category']).mean()
off_peak_averages = off_peak.groupby(['Fuel Category']).mean()

In [107]:
on_peak_averages

,Time Stamp,Gen MW,Day of Week,DayofWeek,DayOfWeek,IsOffPeak
Fuel Category,,,,,,
Dual Fuel,2020-09-15 23:58:05.432078848,4540.257916,1.947912,1.947912,1.947912,0.0
Hydro,2020-09-15 23:58:05.432078848,3305.830427,1.947912,1.947912,1.947912,0.0
Natural Gas,2020-09-15 23:58:05.432078848,4198.489215,1.947912,1.947912,1.947912,0.0
Nuclear,2020-09-15 23:58:05.432078848,3747.539697,1.947912,1.947912,1.947912,0.0
Other Fossil Fuels,2020-09-15 23:58:05.432078848,27.094768,1.947912,1.947912,1.947912,0.0
Other Renewables,2020-09-15 23:58:05.432078848,243.561037,1.947912,1.947912,1.947912,0.0
Wind,2020-09-15 23:58:05.432078848,391.199633,1.947912,1.947912,1.947912,0.0


In [109]:
fig = px.pie(on_peak_averages, values='Gen MW',  names=on_peak_averages.index, title="On-Peak NYISO Fuel Mix")
fig.show()

In [108]:
off_peak_averages

,Time Stamp,Gen MW,Day of Week,DayofWeek,DayOfWeek,IsOffPeak
Fuel Category,,,,,,
Dual Fuel,2020-09-16 02:03:50.455750400,3101.440252,3.806379,3.806379,3.806379,1.0
Hydro,2020-09-16 02:03:50.455750400,2651.194295,3.806379,3.806379,3.806379,1.0
Natural Gas,2020-09-16 02:03:50.455750400,2820.411950,3.806379,3.806379,3.806379,1.0
Nuclear,2020-09-16 02:03:50.455750400,3777.056379,3.806379,3.806379,3.806379,1.0
Other Fossil Fuels,2020-09-16 02:03:50.455750400,2.772911,3.806379,3.806379,3.806379,1.0
Other Renewables,2020-09-16 02:03:50.455750400,236.793801,3.806379,3.806379,3.806379,1.0
Wind,2020-09-16 02:03:50.455750400,480.287062,3.806379,3.806379,3.806379,1.0


In [146]:
fig_off_peak = px.pie(off_peak_averages, values='Gen MW',  names=off_peak_averages.index, title="Off-Peak NYISO Fuel Mix")
fig_off_peak.show()

## Question:

Also answer the following:  What is the difference between base, intermediate, and peaking load?

The difference between base, intermediate, and peaking load is as follow:


*   Base Load: This is the minimum amount of electricity that is needed during the course of the day **Hydro, Nuclear**
*   Intermediate Load: the amount of electricity demand that's higher than base load but lower than peak load: **Natural Gas**
*   Peaking Load: This represent the time of high demand: **Fuel Oil**





In [147]:
import datetime

I used the following source to create the below dictionary. https://docs.python.org/3/tutorial/datastructures.html#dictionaries

In [152]:
electricity_data = {
    "Load Type": ["Base Load", "Intermediate Load", "Peaking Load"],
    "Description": ["Base: Hydro, Nuclear", "Intermediate: Natural Gas", "Peakers: Fuel Oil"],
    "Examples": ["Nuclear, Hydroelectric", "Natural Gas (Combined Cycle), Coal", "Gas Turbines, Diesel Generators"],
    "Usage (MW)": ["Base Load", "Intermediate Load", "Peaking Load"],
    "Date Time": [datetime.datetime.now(), datetime.datetime.now(), datetime.datetime.now()],}



In [153]:
df=pd.DataFrame(electricity_data)

In [154]:
fig = px.bar(df, x="Load Type", y="Usage (MW)", text="Description", title="Electricity Load Types", labels={"y": "MW Usage"}, hover_data=["Date Time"])
fig.update_traces(textposition='outside', hoverinfo="text")
fig.show()
